In [72]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_columns', None)
import os
os.getcwd()

'/Users/cxy/Downloads/Yitiao/MSGO_data'

In [99]:
user = pd.read_excel('user_auction.xlsx')
bid = pd.read_csv('bid_price.csv')
order = pd.read_csv('order.csv')
schedule_labelled = pd.read_excel('schedule_labelled.xls')

In [75]:
print("user shape: ", user.shape)
print("order shape: ", order.shape)
print("bid shape: ", bid.shape)

user shape:  (6733, 33)
order shape:  (430275, 33)
bid shape:  (63665, 19)


# features准备

## 围绕成功拍得拍品的用户

In [76]:
# （1）是否是vip
#     体现在VIP时长中
# （2）成为vip的时长长短
#     已加入userdf
# （3）参与一条拍卖的总场次/总出价参拍次数（不确定是不是由user_bid_order这个变量来表述的）（即 这个用户是不是参与拍卖的老手）
#     schedule_num, bid_num
# （4）该用户最后一次加价（也就是帮助ta赢得拍品的那次加价）距离拍卖截止的时间长短（即 这个用户是不是“最后一刻”冲动决定的） 
#     unknown
# （5）该用户为了该拍品一共出价多少次 每一次出价将价格抬高了几手 每次出价间隔时间（反映该用户多么迫切焦急）  
#     bid_num_per_spu, bid_multiple_per_spu, 间隔时间待计算
# （6）其他人口统计学基本特征
#    user_feature和user_register

In [77]:
userdf = user.copy()
userdf = userdf.rename(columns={'user_id_md5':'user_id'})
userdf.head()

,user_id,user_age,user_gender,user_register_time,user_register_app_id,user_register_utm_source,user_current_vip_level,user_vip_create_time,user_first_order_time,latest_device_app_version,latest_device_os,latest_device_brand,latest_device_model,user_horoscope,user_province,user_city,user_district,user_vip_recently_become_time,user_visit_recently_time,latest_device_os_version,user_latest_order_time,regist_day_page_id,regist_day_page_name,latest_app_time,latest_xcx_time,is_follow_yit,is_follow_my_yit,is_follow_food,is_follow_life,user_real_first_order_app_id,user_real_first_order_utm_source,is_auction_order,is_auction_deposit_order
0,0bd5c6e644f20ce5d596d40055e8f34d,35.0,1,2018-01-26 23:53:13,5.0,NaN,v5,2019-01-03 00:00:00,2018-11-14 21:19:01,NaN,NaN,NaN,NaN,处女座,江西,九江,浔阳区,2019-01-03 17:02:25,2022-03-06 09:34:10.843,NaN,2022-02-17 00:58:37,NaN,NaN,2022-03-06 09:34:10.843,NaN,1,0,0,0,1,3.d2522.6025.4111.562,0,1
1,8330fc2e4780f152915433b24046e659,NaN,1,2018-10-13 12:54:46,5.0,NaN,none,NaT,2018-10-13 12:55:24,4.1.5,ios14.4.2,iPhone,iPhone,NaN,上海,上海,浦东新区,NaT,2022-01-22 12:06:35.188,NaN,2021-12-01 21:36:53,NaN,NaN,NaN,NaN,1,0,0,0,5,NaN,0,1
2,9290c9672da96153dc88dd197ad1cb68,NaN,2,2019-06-02 17:55:04,5.0,9.o1.575,v2,2019-11-14 00:00:00,2019-06-02 18:08:31,NaN,NaN,NaN,NaN,NaN,山东,济南,历下区,2021-03-19 11:36:00,2022-02-24 19:40:49.484,NaN,2022-02-19 18:56:37,p_688,NaN,2022-02-24 19:40:49.484,2022-02-22 13:22:19.178,1,0,0,0,15,9.o1.575,1,1
3,58dae21572fe710a0d493af5bbdeb9e3,NaN,2,2019-12-01 11:31:40,6.0,4.o1.7,none,2021-05-20 17:56:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-20 17:56:38,2022-01-29 13:36:50.854,NaN,2021-04-13 08:12:38,p_4075,NaN,NaN,NaN,1,0,1,0,5,NaN,0,1
4,f766dc6c6bc476b523c5e66c9a15c68b,NaN,1,2020-01-25 15:54:57,5.0,9.o1.679,black_card_vip,2020-03-08 00:00:00,2020-01-25 15:56:46,NaN,NaN,NaN,NaN,NaN,江苏,无锡,锡山区,2021-09-27 08:03:21,2022-03-04 21:16:09.005,NaN,2022-01-26 22:02:01,p_4075,NaN,NaN,2022-02-24 08:30:27.187,1,0,0,0,15,9.o1.679,1,1


（1）是否是vip  
（2）成为vip的时长长短

In [78]:
# function counting days
from datetime import date
def age(create_date):
    today = date.today()
    age = (today.year - create_date.year)*365  + (today.month - create_date.month)*30 - ((today.day) < (create_date.day))
    return age

In [79]:
# convert time to vip days 
userdf.user_vip_create_time = pd.to_datetime(userdf.user_vip_create_time)
userdf['vip_age'] = userdf.user_vip_create_time.apply(age)


In [80]:
# fill nan with mode
userdf.vip_age = userdf[['vip_age']].fillna(userdf['vip_age'].mode()[0])

（3）参与一条拍卖的总场次/总出价参拍次数（不确定是不是由user_bid_order这个变量来表述的）（即 这个用户是不是参与拍卖的老手）

In [81]:
biddf = bid.copy()
biddf.head()

,bid_id,am_activity_id,am_activity_name,schedule_id,schedule_name,user_id,bid_time,spu_id,spu_name,bid_multiple,bid_price,after_bid_price,success_bid,sold_price,order_id,paying_state,user_bid_order,spu_bid_order,auction_order_create_time
0,32662,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,413679edf4207acb9c4ab7c1a061b063,2021-01-09 20:00:32,863442,〓绝版〓80年代 御鹿描金凯旋门拿破仑瓷瓶干邑,1,250.0,250.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:17
1,32663,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,413679edf4207acb9c4ab7c1a061b063,2021-01-09 20:00:44,863339,〓绝版少见〓70年代 詹姆斯马丁20年特级威士忌,1,200.0,200.0,否,未成交,未成交,未成交,2,1,2021-01-10 22:00:05
2,32664,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,4584d43864e9e72d7936572673d3881b,2021-01-09 20:01:57,863323,〓时代经典〓90年代 三得利X.O豪华白兰地 2瓶,1,200.0,200.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:03
3,32665,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,4584d43864e9e72d7936572673d3881b,2021-01-09 20:02:06,863307,〓屡获大奖〓一甲余市樽出原酒威士忌 2瓶,1,200.0,200.0,否,未成交,未成交,未成交,2,1,2021-01-10 22:05:01
4,32666,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,8955ffba44c6336debf8319706e0904d,2021-01-09 20:02:27,863448,〓御殿场限定版〓富士山麓、樽熟原酒50°威士忌 2瓶,1,200.0,1200.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:17


In [82]:
# bid number for per user
bid_num = biddf.groupby(['user_id'])['bid_id'].count().reset_index()
bid_num = bid_num.rename(columns={'bid_id':'bid_num'})
bid_num

,user_id,bid_num
0,00045953c87229eba0cfb93ce290e8d9,1
1,000df6b9543e319e0cae8e594fafd9c5,3
2,000eaf1ff4a8eba6f21a91d86f42d372,1
3,001de79a499b7563944e46aee9fe785d,1
4,002c38df71f32e3955508f2713302d91,3
...,...,...
5778,ffbd2aba7dacfbfb41d335442953e4d3,3
5779,ffcce5993e48bc20e614f390a0fdeaaf,1
5780,ffcf27513bbf54b28b22f72637a5fe0a,7
5781,ffd6441c9f2c5f425093ab50b91548b9,2


In [83]:
# schedule number for per user
schedule_num = biddf.groupby('user_id')['schedule_id'].nunique().reset_index()
schedule_num = schedule_num.rename(columns={'schedule_id':'schedule_num'})
schedule_num

,user_id,schedule_num
0,00045953c87229eba0cfb93ce290e8d9,1
1,000df6b9543e319e0cae8e594fafd9c5,1
2,000eaf1ff4a8eba6f21a91d86f42d372,1
3,001de79a499b7563944e46aee9fe785d,1
4,002c38df71f32e3955508f2713302d91,1
...,...,...
5778,ffbd2aba7dacfbfb41d335442953e4d3,2
5779,ffcce5993e48bc20e614f390a0fdeaaf,1
5780,ffcf27513bbf54b28b22f72637a5fe0a,1
5781,ffd6441c9f2c5f425093ab50b91548b9,1


（4）该用户最后一次加价（也就是帮助ta赢得拍品的那次加价）距离拍卖截止的时间长短（即 这个用户是不是“最后一刻”冲动决定的）

In [84]:
# TBD

（5）该用户为了该拍品一共出价多少次 每一次出价将价格抬高了几手 每次出价间隔时间（反映该用户多么迫切焦急）

In [85]:
biddf.head()

,bid_id,am_activity_id,am_activity_name,schedule_id,schedule_name,user_id,bid_time,spu_id,spu_name,bid_multiple,bid_price,after_bid_price,success_bid,sold_price,order_id,paying_state,user_bid_order,spu_bid_order,auction_order_create_time
0,32662,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,413679edf4207acb9c4ab7c1a061b063,2021-01-09 20:00:32,863442,〓绝版〓80年代 御鹿描金凯旋门拿破仑瓷瓶干邑,1,250.0,250.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:17
1,32663,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,413679edf4207acb9c4ab7c1a061b063,2021-01-09 20:00:44,863339,〓绝版少见〓70年代 詹姆斯马丁20年特级威士忌,1,200.0,200.0,否,未成交,未成交,未成交,2,1,2021-01-10 22:00:05
2,32664,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,4584d43864e9e72d7936572673d3881b,2021-01-09 20:01:57,863323,〓时代经典〓90年代 三得利X.O豪华白兰地 2瓶,1,200.0,200.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:03
3,32665,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,4584d43864e9e72d7936572673d3881b,2021-01-09 20:02:06,863307,〓屡获大奖〓一甲余市樽出原酒威士忌 2瓶,1,200.0,200.0,否,未成交,未成交,未成交,2,1,2021-01-10 22:05:01
4,32666,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,8955ffba44c6336debf8319706e0904d,2021-01-09 20:02:27,863448,〓御殿场限定版〓富士山麓、樽熟原酒50°威士忌 2瓶,1,200.0,1200.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:17


In [86]:
# bid number for per user in each spu
bid_num_per_spu = biddf.groupby(['user_id','spu_id'])['bid_id'].count().reset_index()
bid_num_per_spu = bid_num_per_spu.rename(columns={'bid_id':'bid_num_u_s'})
bid_num_per_spu

,user_id,spu_id,bid_num_u_s
0,00045953c87229eba0cfb93ce290e8d9,1067626,1
1,000df6b9543e319e0cae8e594fafd9c5,929065,3
2,000eaf1ff4a8eba6f21a91d86f42d372,864498,1
3,001de79a499b7563944e46aee9fe785d,903064,1
4,002c38df71f32e3955508f2713302d91,1018195,3
...,...,...,...
39812,fffab188187b5b0d5ad47dff191b77ea,994322,1
39813,fffab188187b5b0d5ad47dff191b77ea,994354,1
39814,fffab188187b5b0d5ad47dff191b77ea,1022106,1
39815,fffab188187b5b0d5ad47dff191b77ea,1051477,2


In [87]:
# bid multiple for per user in each spu
bid_multiple_per_spu = biddf.groupby(['user_id','spu_id'])['bid_multiple'].sum().reset_index()
bid_multiple_per_spu = bid_multiple_per_spu.rename(columns={'bid_multiple':'bid_multiple_num'})
bid_multiple_per_spu

,user_id,spu_id,bid_multiple_num
0,00045953c87229eba0cfb93ce290e8d9,1067626,1
1,000df6b9543e319e0cae8e594fafd9c5,929065,3
2,000eaf1ff4a8eba6f21a91d86f42d372,864498,1
3,001de79a499b7563944e46aee9fe785d,903064,3
4,002c38df71f32e3955508f2713302d91,1018195,3
...,...,...,...
39812,fffab188187b5b0d5ad47dff191b77ea,994322,1
39813,fffab188187b5b0d5ad47dff191b77ea,994354,1
39814,fffab188187b5b0d5ad47dff191b77ea,1022106,1
39815,fffab188187b5b0d5ad47dff191b77ea,1051477,2


（6）其他人口统计学基本特征

In [88]:
userdf.head()

,user_id,user_age,user_gender,user_register_time,user_register_app_id,user_register_utm_source,user_current_vip_level,user_vip_create_time,user_first_order_time,latest_device_app_version,latest_device_os,latest_device_brand,latest_device_model,user_horoscope,user_province,user_city,user_district,user_vip_recently_become_time,user_visit_recently_time,latest_device_os_version,user_latest_order_time,regist_day_page_id,regist_day_page_name,latest_app_time,latest_xcx_time,is_follow_yit,is_follow_my_yit,is_follow_food,is_follow_life,user_real_first_order_app_id,user_real_first_order_utm_source,is_auction_order,is_auction_deposit_order,vip_age
0,0bd5c6e644f20ce5d596d40055e8f34d,35.0,1,2018-01-26 23:53:13,5.0,NaN,v5,2019-01-03 00:00:00,2018-11-14 21:19:01,NaN,NaN,NaN,NaN,处女座,江西,九江,浔阳区,2019-01-03 17:02:25,2022-03-06 09:34:10.843,NaN,2022-02-17 00:58:37,NaN,NaN,2022-03-06 09:34:10.843,NaN,1,0,0,0,1,3.d2522.6025.4111.562,0,1,1185.0
1,8330fc2e4780f152915433b24046e659,NaN,1,2018-10-13 12:54:46,5.0,NaN,none,NaT,2018-10-13 12:55:24,4.1.5,ios14.4.2,iPhone,iPhone,NaN,上海,上海,浦东新区,NaT,2022-01-22 12:06:35.188,NaN,2021-12-01 21:36:53,NaN,NaN,NaN,NaN,1,0,0,0,5,NaN,0,1,334.0
2,9290c9672da96153dc88dd197ad1cb68,NaN,2,2019-06-02 17:55:04,5.0,9.o1.575,v2,2019-11-14 00:00:00,2019-06-02 18:08:31,NaN,NaN,NaN,NaN,NaN,山东,济南,历下区,2021-03-19 11:36:00,2022-02-24 19:40:49.484,NaN,2022-02-19 18:56:37,p_688,NaN,2022-02-24 19:40:49.484,2022-02-22 13:22:19.178,1,0,0,0,15,9.o1.575,1,1,885.0
3,58dae21572fe710a0d493af5bbdeb9e3,NaN,2,2019-12-01 11:31:40,6.0,4.o1.7,none,2021-05-20 17:56:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-20 17:56:38,2022-01-29 13:36:50.854,NaN,2021-04-13 08:12:38,p_4075,NaN,NaN,NaN,1,0,1,0,5,NaN,0,1,334.0
4,f766dc6c6bc476b523c5e66c9a15c68b,NaN,1,2020-01-25 15:54:57,5.0,9.o1.679,black_card_vip,2020-03-08 00:00:00,2020-01-25 15:56:46,NaN,NaN,NaN,NaN,NaN,江苏,无锡,锡山区,2021-09-27 08:03:21,2022-03-04 21:16:09.005,NaN,2022-01-26 22:02:01,p_4075,NaN,NaN,2022-02-24 08:30:27.187,1,0,0,0,15,9.o1.679,1,1,760.0


In [89]:
userdf.shape

(6733, 34)

In [90]:
userdf.isna().sum()

user_id                                0
user_age                            4830
user_gender                            0
user_register_time                     0
user_register_app_id                 793
user_register_utm_source            3518
user_current_vip_level                 0
user_vip_create_time                2377
user_first_order_time               2091
latest_device_app_version           5208
latest_device_os                    5208
latest_device_brand                 5208
latest_device_model                 5208
user_horoscope                      4830
user_province                       1498
user_city                           1498
user_district                       1574
user_vip_recently_become_time       2435
user_visit_recently_time              12
latest_device_os_version            6733
user_latest_order_time                 0
regist_day_page_id                  3935
regist_day_page_name                4554
latest_app_time                     4817
latest_xcx_time 

In [91]:
user_feature = userdf[['user_id','user_gender','user_province']]
user_feature.head()

,user_id,user_gender,user_province
0,0bd5c6e644f20ce5d596d40055e8f34d,1,江西
1,8330fc2e4780f152915433b24046e659,1,上海
2,9290c9672da96153dc88dd197ad1cb68,2,山东
3,58dae21572fe710a0d493af5bbdeb9e3,2,NaN
4,f766dc6c6bc476b523c5e66c9a15c68b,1,江苏


In [92]:
user_register = userdf[['user_id','user_register_time','user_register_app_id','vip_age']]
user_register.head()

,user_id,user_register_time,user_register_app_id,vip_age
0,0bd5c6e644f20ce5d596d40055e8f34d,2018-01-26 23:53:13,5.0,1185.0
1,8330fc2e4780f152915433b24046e659,2018-10-13 12:54:46,5.0,334.0
2,9290c9672da96153dc88dd197ad1cb68,2019-06-02 17:55:04,5.0,885.0
3,58dae21572fe710a0d493af5bbdeb9e3,2019-12-01 11:31:40,6.0,334.0
4,f766dc6c6bc476b523c5e66c9a15c68b,2020-01-25 15:54:57,5.0,760.0


In [93]:
# convert time to registry days 
user_register.user_register_time = pd.to_datetime(user_register.user_register_time)
user_register['user_refister_age'] = user_register.user_register_time.apply(age)
user_register.head()

/Users/cxy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/cxy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,user_register_time,user_register_app_id,vip_age,user_refister_age
0,0bd5c6e644f20ce5d596d40055e8f34d,2018-01-26 23:53:13,5.0,1185.0,1549
1,8330fc2e4780f152915433b24046e659,2018-10-13 12:54:46,5.0,334.0,1280
2,9290c9672da96153dc88dd197ad1cb68,2019-06-02 17:55:04,5.0,885.0,1035
3,58dae21572fe710a0d493af5bbdeb9e3,2019-12-01 11:31:40,6.0,334.0,855
4,f766dc6c6bc476b523c5e66c9a15c68b,2020-01-25 15:54:57,5.0,760.0,819


In [94]:
user_register = user_register.drop(columns = ['user_register_time'])

In [95]:
user_register.head()

,user_id,user_register_app_id,vip_age,user_refister_age
0,0bd5c6e644f20ce5d596d40055e8f34d,5.0,1185.0,1549
1,8330fc2e4780f152915433b24046e659,5.0,334.0,1280
2,9290c9672da96153dc88dd197ad1cb68,5.0,885.0,1035
3,58dae21572fe710a0d493af5bbdeb9e3,6.0,334.0,855
4,f766dc6c6bc476b523c5e66c9a15c68b,5.0,760.0,819


## 围绕被逃逸者放弃的拍品

In [96]:
# 围绕被逃逸者放弃的拍品
# （1）产品品类  
# （2）最终成交价相较于最初定价的涨幅 或者也就是说 一共被全体参拍者加了多少手价  
#     price_gap['increase_rate']

In [97]:
#（1）产品品类

In [119]:
schedule_labelled = schedule_labelled[['schedule_id','category_label']]
schedule_labelled.head()

,schedule_id,category_label
0,1006976,wine
1,1006977,wine
2,1006997,tea
3,1007111,tea
4,1007114,painting


In [103]:
# increase rate

In [104]:
price_gap = biddf.groupby('spu_id').agg(min_price = ('after_bid_price', 'min'),
                                           max_price = ('after_bid_price', 'max'),
                                           user_num = ('user_id', 'size')).reset_index()
price_gap['gap'] = price_gap['max_price'] - price_gap['min_price']
price_gap['increase_rate'] = (price_gap['max_price'] - price_gap['min_price'])/price_gap['min_price']

price_gap.head()

,spu_id,min_price,max_price,user_num,gap,increase_rate
0,863303,200.0,3200.0,14,3000.0,15.000000
1,863305,5750.0,7000.0,6,1250.0,0.217391
2,863307,200.0,2800.0,14,2600.0,13.000000
3,863309,2300.0,3200.0,4,900.0,0.391304
4,863311,1500.0,3300.0,7,1800.0,1.200000


In [105]:
price_gap = price_gap[['spu_id','increase_rate']]

In [106]:
price_gap

,spu_id,increase_rate
0,863303,15.000000
1,863305,0.217391
2,863307,13.000000
3,863309,0.391304
4,863311,1.200000
...,...,...
11202,1073448,0.000000
11203,1073452,0.000000
11204,1073462,1.000000
11205,1073466,0.000000


## 围绕拍卖机制

In [107]:
# （1）该拍卖场次设定的保证金金额 
#     unknown
# （2）本场全部参拍拍品的总数  
#     spu_num
# （3）系统设定的每一手加价是加多少钱（不同拍卖场次/拍品品类可能不一样）
#     已经加入biddf

In [108]:
biddf.head()

,bid_id,am_activity_id,am_activity_name,schedule_id,schedule_name,user_id,bid_time,spu_id,spu_name,bid_multiple,bid_price,after_bid_price,success_bid,sold_price,order_id,paying_state,user_bid_order,spu_bid_order,auction_order_create_time
0,32662,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,413679edf4207acb9c4ab7c1a061b063,2021-01-09 20:00:32,863442,〓绝版〓80年代 御鹿描金凯旋门拿破仑瓷瓶干邑,1,250.0,250.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:17
1,32663,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,413679edf4207acb9c4ab7c1a061b063,2021-01-09 20:00:44,863339,〓绝版少见〓70年代 詹姆斯马丁20年特级威士忌,1,200.0,200.0,否,未成交,未成交,未成交,2,1,2021-01-10 22:00:05
2,32664,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,4584d43864e9e72d7936572673d3881b,2021-01-09 20:01:57,863323,〓时代经典〓90年代 三得利X.O豪华白兰地 2瓶,1,200.0,200.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:03
3,32665,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,4584d43864e9e72d7936572673d3881b,2021-01-09 20:02:06,863307,〓屡获大奖〓一甲余市樽出原酒威士忌 2瓶,1,200.0,200.0,否,未成交,未成交,未成交,2,1,2021-01-10 22:05:01
4,32666,1013349,珍稀佳酿 威士忌及白兰地专场,1006976,第一场,8955ffba44c6336debf8319706e0904d,2021-01-09 20:02:27,863448,〓御殿场限定版〓富士山麓、樽熟原酒50°威士忌 2瓶,1,200.0,1200.0,否,未成交,未成交,未成交,1,1,2021-01-10 22:00:17


In [109]:
#（1）该拍卖场次设定的保证金金额不可知

In [110]:
#（2）本场全部参拍拍品的总数
# spu in each schedule
spu_num = biddf.groupby(['schedule_id'])['spu_id'].nunique().reset_index()
spu_num = spu_num.rename(columns={'spu_id':'spu_number'})
spu_num

,schedule_id,spu_number
0,1006976,50
1,1006977,50
2,1006997,28
3,1007111,29
4,1007112,48
...,...,...
377,1012766,17
378,1012767,18
379,1012769,11
380,1012776,1


In [111]:
#（3）系统设定的每一手加价是加多少钱（不同拍卖场次/拍品品类可能不一样）
biddf['sho'] = biddf.bid_price/biddf.bid_multiple

# Selection & Merge

In [112]:
# select rows with 
modeldf = biddf.loc[biddf['paying_state'].isin(['PAID','TIMEOUT_NON_PAYMENT'])]

In [113]:
modeldf.paying_state.value_counts()

PAID                   7607
TIMEOUT_NON_PAYMENT    3532
Name: paying_state, dtype: int64

In [114]:
modeldf['paying_state'] = modeldf.paying_state.apply(lambda x: 0 if x == 'TIMEOUT_NON_PAYMENT' else 1)


/Users/cxy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
modeldf.paying_state.value_counts()

1    7607
0    3532
Name: paying_state, dtype: int64

In [116]:
# merge user infomation together
print('before: ', modeldf.shape)
modeldf = pd.merge(modeldf, schedule_num, how = 'left', on=['user_id'])
modeldf = pd.merge(modeldf, bid_num, how = 'left', on=['user_id'])
modeldf = pd.merge(modeldf, bid_num_per_spu, how = 'left', on=['user_id','spu_id'])
modeldf = pd.merge(modeldf, bid_multiple_per_spu, how = 'left', on=['user_id','spu_id'])
modeldf = pd.merge(modeldf, user_feature, how = 'left', on=['user_id'])
modeldf = pd.merge(modeldf, user_register, how = 'left', on=['user_id'])
print('before: ', modeldf.shape)
# expect to be (11139, 20), (11139, 29)

before:  (11139, 20)
before:  (11139, 29)


In [117]:
# merge SPU infomation together

In [118]:
print('before: ', modeldf.shape)
modeldf = pd.merge(modeldf, price_gap, how = 'left', on=['spu_id'])
print('after: ', modeldf.shape)
# expect to be (11139, 29), (11139, 30)

before:  (11139, 29)
after:  (11139, 30)


In [120]:
# merge auction infomation together

In [121]:
print('before: ', modeldf.shape)
modeldf = pd.merge(modeldf, spu_num, how = 'left', on=['schedule_id'])
modeldf = pd.merge(modeldf, schedule_labelled, how = 'left', on=['schedule_id'])
print('after: ', modeldf.shape)
# expect to be (11139, 30), (11139, 32)

before:  (11139, 30)
after:  (11139, 32)


In [122]:
# yeahhhh
mergeddf = modeldf.copy()

In [123]:
mergeddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11139 entries, 0 to 11138
Data columns (total 32 columns):
bid_id                       11139 non-null int64
am_activity_id               11139 non-null int64
am_activity_name             11139 non-null object
schedule_id                  11139 non-null int64
schedule_name                11139 non-null object
user_id                      11139 non-null object
bid_time                     11139 non-null object
spu_id                       11139 non-null int64
spu_name                     11139 non-null object
bid_multiple                 11139 non-null int64
bid_price                    11139 non-null float64
after_bid_price              11139 non-null float64
success_bid                  11139 non-null object
sold_price                   11139 non-null object
order_id                     11139 non-null object
paying_state                 11139 non-null int64
user_bid_order               11139 non-null int64
spu_bid_order                

In [124]:
data_prep = mergeddf[['bid_multiple', 'bid_price', 'after_bid_price', 
                      'paying_state', 'sho', 'schedule_num', 
                      'bid_num', 'bid_num_u_s', 'bid_multiple_num', 
                      'user_gender', 'user_province', 'user_register_app_id', 
                      'vip_age', 'increase_rate','spu_number', 
                      'category_label']]

In [125]:
data_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11139 entries, 0 to 11138
Data columns (total 16 columns):
bid_multiple            11139 non-null int64
bid_price               11139 non-null float64
after_bid_price         11139 non-null float64
paying_state            11139 non-null int64
sho                     11139 non-null float64
schedule_num            11139 non-null int64
bid_num                 11139 non-null int64
bid_num_u_s             11139 non-null int64
bid_multiple_num        11139 non-null int64
user_gender             11139 non-null int64
user_province           10672 non-null object
user_register_app_id    9871 non-null float64
vip_age                 11139 non-null float64
increase_rate           11139 non-null float64
spu_number              11139 non-null int64
category_label          11139 non-null object
dtypes: float64(6), int64(8), object(2)
memory usage: 1.4+ MB


In [126]:
data_prep.isna().sum()

bid_multiple               0
bid_price                  0
after_bid_price            0
paying_state               0
sho                        0
schedule_num               0
bid_num                    0
bid_num_u_s                0
bid_multiple_num           0
user_gender                0
user_province            467
user_register_app_id    1268
vip_age                    0
increase_rate              0
spu_number                 0
category_label             0
dtype: int64

In [127]:
# fill nan vip age with mode
data_prep.vip_age = data_prep[['vip_age']].fillna(data_prep['vip_age'].mode()[0])

In [128]:
# fill nan province with '未知'
data_prep.user_province = data_prep[['user_province']].fillna('未知')

In [129]:
# fill nan province with 'unkown_channel'
data_prep.user_register_app_id = data_prep[['user_register_app_id']].fillna('unkown_channel')

In [130]:
data_prep.isna().sum()

bid_multiple            0
bid_price               0
after_bid_price         0
paying_state            0
sho                     0
schedule_num            0
bid_num                 0
bid_num_u_s             0
bid_multiple_num        0
user_gender             0
user_province           0
user_register_app_id    0
vip_age                 0
increase_rate           0
spu_number              0
category_label          0
dtype: int64

In [131]:
data = pd.get_dummies(data_prep)
data.head()

,bid_multiple,bid_price,after_bid_price,paying_state,sho,schedule_num,bid_num,bid_num_u_s,bid_multiple_num,user_gender,vip_age,increase_rate,spu_number,user_province_上海,user_province_云南,user_province_内蒙古,user_province_北京,user_province_吉林,user_province_四川,user_province_天津,user_province_宁夏,user_province_安徽,user_province_山东,user_province_山西,user_province_广东,user_province_广西,user_province_新疆,user_province_未知,user_province_江苏,user_province_江西,user_province_河北,user_province_河南,user_province_浙江,user_province_海南,user_province_湖北,user_province_湖南,user_province_甘肃,user_province_福建,user_province_西藏,user_province_贵州,user_province_辽宁,user_province_重庆,user_province_陕西,user_province_青海,user_province_黑龙江,user_register_app_id_1.0,user_register_app_id_2.0,user_register_app_id_3.0,user_register_app_id_5.0,user_register_app_id_6.0,user_register_app_id_7.0,user_register_app_id_15.0,user_register_app_id_20.0,user_register_app_id_26.0,user_register_app_id_33.0,user_register_app_id_43.0,user_register_app_id_52.0,user_register_app_id_53.0,user_register_app_id_unkown_channel,category_label_ceramic,category_label_fashion,category_label_fragrance,category_label_jade,category_label_jewelry,category_label_luxury,category_label_others,category_label_painting,category_label_sports,category_label_tea,category_label_wine
0,1,300.0,7600.0,1,300.0,7,64,1,1,1,1250.0,0.041096,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,400.0,11100.0,1,400.0,2,59,2,2,1,334.0,0.121212,50,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,400.0,23800.0,1,400.0,2,17,1,1,1,1429.0,0.017094,50,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,200.0,2400.0,1,200.0,3,29,1,1,1,489.0,1.000000,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,200.0,2200.0,1,200.0,2,59,1,1,1,334.0,0.833333,50,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [132]:
data.paying_state.value_counts()

1    7607
0    3532
Name: paying_state, dtype: int64

In [133]:
X = data.drop(columns=['paying_state'])
y = data.paying_state

In [134]:
from sklearn.model_selection import train_test_split

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
y_test

6117    1
483     1
8413    1
9900    1
9412    1
       ..
4907    1
5325    1
9693    1
8024    1
8851    1
Name: paying_state, Length: 3342, dtype: int64

# Decision Tree with depth = 10

In [135]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree     import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [136]:
model_dt = DecisionTreeClassifier(criterion='entropy', max_depth=10)

# Train model using training dataset
model_dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [137]:
y_pred_dt = model_dt.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))
print("Accuracy:",accuracy_score(y_test, y_pred_dt, normalize=True, sample_weight=None))
print("Precision:",precision_score(y_test, y_pred_dt, pos_label=0, sample_weight=None))
print("Recall:",recall_score(y_test, y_pred_dt, pos_label=0, sample_weight=None))

Confusion Matrix:
 [[ 968  118]
 [  84 2172]]
Accuracy: 0.9395571514063435
Precision: 0.9201520912547528
Recall: 0.8913443830570903


# Random Forest

In [138]:
# Ramdom Forests (base estimator: decision tree)
# n estimator : n regression trees
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

mod_rf2 = RandomForestClassifier()
mod_rf2.get_params()

params_rf = {
  "n_estimators": [400, 800],
  "max_depth": [4, 8, 12],
  "min_samples_leaf": [0.05, 0.1]
}

rf_gridsearch = GridSearchCV(
  estimator = mod_rf2,
  param_grid = params_rf,
  scoring = "accuracy",
  cv = 5,
  n_jobs = -1
)

rf_gridsearch.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [139]:
rf_gridsearch.best_params_

{'max_depth': 8, 'min_samples_leaf': 0.05, 'n_estimators': 800}

In [140]:
rf_best_gridsearch = rf_gridsearch.best_estimator_
y_pred_rf_gridsearch = rf_best_gridsearch.predict(X_test)
accuracy_score(y_test, y_pred_rf_gridsearch)

0.7887492519449432

In [141]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf_gridsearch))
print("Accuracy:",accuracy_score(y_test, y_pred_rf_gridsearch, normalize=True, sample_weight=None))
print("Precision:",precision_score(y_test, y_pred_rf_gridsearch, pos_label=0, sample_weight=None))
print("Recall:",recall_score(y_test, y_pred_rf_gridsearch, pos_label=0, sample_weight=None))

Confusion Matrix:
 [[ 459  627]
 [  79 2177]]
Accuracy: 0.7887492519449432
Precision: 0.8531598513011153
Recall: 0.42265193370165743
